In [1]:
%load_ext autoreload
%autoreload 2
!pwd
!hostname

/net/scratch/ajasja/projects/truncator/R4_crystal_nMS_Y2H
dig3


In [2]:
import numpy as np
from glob import glob
import numpy as np
import os
import truncator
import pandas as pd
import truncator

In [3]:
#structs = truncator.read_file_lines("/home/ajasja/struct/ZCON/nMS.txt", trim=True)
#for s in structs:
#    !cp /home/ajasja/struct/ZCON/{s}  /home/ajasja/struct/ZCON/nMS

In [4]:
xtal_structs = truncator.read_file_lines("/home/ajasja/struct/ZCON/xtal_no_duplicate_bases.txt", trim=True)
for s in xtal_structs:
    !cp /home/ajasja/struct/ZCON/{s}  data/
    file_name = f"/home/ajasja/struct/ZCON/{s}"
    if not os.path.isfile(file_name):
        print ("NOT FOUND: ",file_name)

In [5]:
nMS_structs = truncator.read_file_lines("/home/ajasja/struct/ZCON/nMS_no_duplicate_base.txt", trim=True)
for s in nMS_structs:
    !cp /home/ajasja/struct/ZCON/{s}  data/
    file_name = f"/home/ajasja/struct/ZCON/{s}"
    if not os.path.isfile(file_name):
        print ("NOT FOUND: ",file_name)

In [6]:
Y2H_structs = truncator.read_file_lines("/home/ajasja/struct/ZCON/Y2H_no_duplicate_base.txt", trim=True)
for s in Y2H_structs:
    !cp /home/ajasja/struct/ZCON/{s}  data/
    file_name = f"/home/ajasja/struct/ZCON/{s}"
    if not os.path.isfile(file_name):
        print ("NOT FOUND: ",file_name)

In [8]:
structs = glob("data/*.pdb")
len(structs)

126

In [8]:


#truncator.cut_bundles(struct_name=structs[0], out_dir='out/R1/cut/4H', num_heptads=4, )
truncator.cut_bundles(struct_name=structs[0], out_dir='out/test/cut/4H', num_heptads=4, )

 PyMOL not running, entering library mode (experimental)


['out/test/cut/4H/ZCON_94__numH4__from-21.54__to16.85.pdb',
 'out/test/cut/4H/ZCON_94__numH4__from-19.44__to18.95.pdb',
 'out/test/cut/4H/ZCON_94__numH4__from-17.33__to21.05.pdb',
 'out/test/cut/4H/ZCON_94__numH4__from-15.23__to23.16.pdb',
 'out/test/cut/4H/ZCON_94__numH4__from-13.13__to25.26.pdb']

In [12]:
out_dir_base = "out/R1"
heptades_lengths = [2.5,3,3.5,4,4.5,5]
#!rm -rf {out_dir_base}

In [14]:
for l in heptades_lengths:
    for struct in structs:
        try:
            truncator.cut_bundles(struct_name=struct, out_dir=f'{out_dir_base}/01_cut/{l:1.1f}H', num_heptads=l)
        except Exception as e:
            print (struct, l, e)

/net/scratch/ajasja/projects/truncator/R4_crystal_nMS_Y2H/truncator/cutting.py:77: RuntimeWarning: divide by zero encountered in true_divide
  bottoms = np.arange(highest_bottom, lowest_top, step_delta_z)


data/ZCON_9_FL_CC_trim.pdb 2.5 Maximum allowed size exceeded
data/ZCON_VR_4.pdb 2.5 'NoneType' object is not subscriptable
data/ZCON_16.pdb 2.5 too many indices for array
data/ZCON_VR_5.pdb 2.5 'NoneType' object is not subscriptable
data/ZCON_9_FL_CC_trim.pdb 3 Maximum allowed size exceeded
data/ZCON_VR_4.pdb 3 'NoneType' object is not subscriptable
data/ZCON_16.pdb 3 too many indices for array
data/ZCON_VR_5.pdb 3 'NoneType' object is not subscriptable
data/ZCON_9_FL_CC_trim.pdb 3.5 Maximum allowed size exceeded
data/ZCON_VR_4.pdb 3.5 'NoneType' object is not subscriptable
data/ZCON_16.pdb 3.5 too many indices for array
data/ZCON_VR_5.pdb 3.5 'NoneType' object is not subscriptable
data/ZCON_9_FL_CC_trim.pdb 4 Maximum allowed size exceeded
data/ZCON_VR_4.pdb 4 'NoneType' object is not subscriptable
data/ZCON_16.pdb 4 too many indices for array
data/ZCON_VR_5.pdb 4 'NoneType' object is not subscriptable
data/ZCON_9_FL_CC_trim.pdb 4.5 Maximum allowed size exceeded
data/ZCON_VR_4.pdb 4.5 

In [16]:
groups = (("AB", "CD"), ("AD", "BC"), ("A", "BCD"), ("B", "ACD"), ("C", "ABD"), ("D", "ABC"))

In [18]:
%%time
for l in heptades_lengths:
    all_cut = glob(f"{out_dir_base}/01_cut/{l:1.1f}H/*.pdb")
    for struct in all_cut:
        for group_a, group_b in groups:
            try:
                truncator.regroup_chains(struct, f"{out_dir_base}/02_regruped/{l:1.1f}H",
                                new_chain_A=group_a, new_chain_B=group_b)
            except Exception as e:
                print (struct, l, e, group_a, group_b)

out/R1/01_cut/2.5H/ZCON_39__numH2.5__from-2.66__to20.42.pdb 2.5 Chains: {'E', 'F'} have not been specified new_chain_A and new_chain_B AB CD
out/R1/01_cut/2.5H/ZCON_39__numH2.5__from-2.66__to20.42.pdb 2.5 Chains: {'E', 'F'} have not been specified new_chain_A and new_chain_B AD BC
out/R1/01_cut/2.5H/ZCON_39__numH2.5__from-2.66__to20.42.pdb 2.5 Chains: {'E', 'F'} have not been specified new_chain_A and new_chain_B A BCD
out/R1/01_cut/2.5H/ZCON_39__numH2.5__from-2.66__to20.42.pdb 2.5 Chains: {'E', 'F'} have not been specified new_chain_A and new_chain_B B ACD
out/R1/01_cut/2.5H/ZCON_39__numH2.5__from-2.66__to20.42.pdb 2.5 Chains: {'E', 'F'} have not been specified new_chain_A and new_chain_B C ABD
out/R1/01_cut/2.5H/ZCON_39__numH2.5__from-2.66__to20.42.pdb 2.5 Chains: {'E', 'F'} have not been specified new_chain_A and new_chain_B D ABC
out/R1/01_cut/2.5H/ZCON_39__numH2.5__from-20.54__to02.54.pdb 2.5 Chains: {'E', 'F'} have not been specified new_chain_A and new_chain_B AB CD
out/R1/01_cu

In [7]:
!pwd

/net/scratch/ajasja/projects/truncator/R4_crystal_nMS_Y2H


In [17]:
%%time
all_regrouped = glob(f"{out_dir_base}/02_regruped/4.5H/*.pdb")            
truncator.score_interface(all_regrouped[0],
                    script_name="truncator/xml/05_specific_interface_markers.xml",
                    out_dir="out/test/03_score_new")

/software/rosetta/latest/bin/rosetta_scripts     -out:path:all out/test/03_score_new     -out:file:o  ZCON_70__numH4.5__from-20.46__to18.79__grAB-CD.pdb     -out:file:scorefile  ZCON_70__numH4.5__from-20.46__to18.79__grAB-CD.sc       -parser:protocol truncator/xml/05_specific_interface_markers.xml -s out/R1/02_regruped/4.5H/ZCON_70__numH4.5__from-20.46__to18.79__grAB-CD.pdb     -print_pymol_selection 0     -beta     -in:file:fullatom     -renumber_pdb 1     -overwrite     -out:file:scorefile_format json     -holes:dalphaball truncator/scripts/DAlphaBall.gcc -out:no_nstruct_label     > out/test/03_score_new/ZCON_70__numH4.5__from-20.46__to18.79__grAB-CD.log
Saving to: /net/scratch/ajasja/projects/truncator/R4_crystal_nMS_Y2H/out/test/03_score_new/ZCON_70__numH4.5__from-20.46__to18.79__grAB-CD_IA.sc
CPU times: user 146 ms, sys: 12.8 ms, total: 159 ms
Wall time: 35 s


In [10]:
import dask
from dask.distributed import Client, progress
client = Client(scheduler_file='scheduler.json'); client

Client Scheduler: tcp://172.16.131.33:13037 Dashboard: http://172.16.131.33:8787/status,Cluster Workers: 20 Cores: 20 Memory: 40.00 GB


In [13]:
all_cut = glob(f"{out_dir_base}/02_regruped/*H/*.pdb")
len(all_cut)

27336

In [14]:
res = []
for l in heptades_lengths:
    all_cut = glob(f"{out_dir_base}/02_regruped/{l:1.1f}H/*.pdb")
    for struct in all_cut:
        res.append(client.submit(truncator.score_interface, struct,
                script_name="truncator/xml/05_specific_interface_markers.xml",
                out_dir=f"{out_dir_base}/03_score_SASA_SEQ/{l:1.1f}H", test_run=False, skip_existing=True))

In [15]:
#run only to cancel
#for r in res:
#    r.cancel()

In [14]:
#res[0].cancel()

In [14]:
print(len(res))
import dask
dask.distributed.wait(res, timeout=5);

27336


TimeoutError: Timeout

In [16]:
retry_futures = []
for k, v in client.who_has().items():
    future = dask.distributed.Future(key=k, client=client, inform=True, state=None)
    if future.status == "error":
        retry_futures.append(future)
        

In [18]:
client.retry(retry_futures)

In [20]:
###Do reloop
def do_reloop(struct_name):
    reloop = truncator.reloop(struct_name, "out/R2/relooped/random", 
                          rosetta_bin="/home/brunette/src/Rosetta_dev/main/source/bin/rosetta_scripts.hdf5.linuxgccrelease",
                          script_name="truncator/xml/12_redesign_loops_tl_gen_profile_SymAnn.xml",
                            test_run=False, skip_existing=True)
    return reloop

In [22]:

structs = glob("out/R2/cut/3.0H/*.pdb"); structs
rstructs = np.random.choice(structs, 100); 
rstructs = sorted(set(rstructs)); 
rstructs;

In [23]:
res = client.map(do_reloop, rstructs)
results = client.gather(res)

In [24]:
lstructs = truncator.find_no_output_pdbs("out/R2/relooped/random", ignore_unfinished=True) 
all_struct = glob("out/R2/cut/*/*.pdb")
lstructs = [truncator.find_input(os.path.basename(struct), all_struct) for struct in lstructs]

In [25]:
import sys
def do_reloop_longer(struct_name):
    reloop = truncator.reloop(struct_name, "out/R2/relooped/random_longer_loops", 
                          rosetta_bin="/home/brunette/src/Rosetta_dev/main/source/bin/rosetta_scripts.hdf5.linuxgccrelease",
                          script_name="truncator/xml/13_redesign_loops_tl_gen_profile_SymAnn_longerloops.xml",
                            test_run=False, skip_existing=True)
    return reloop
    


In [26]:
res = client.map(do_reloop_longer, lstructs)
results = client.gather(res)

KilledWorker: ('do_reloop_longer-a3bea14f4fc1223abd43ce9f98c8fe84', 'tcp://172.16.131.59:29787')

In [ ]:
truncator.regroup_chains("testing_out//ZCON_37_3+1__numH4__from-22.25__to17.45.pdb", "testing_out/grouped",
                        new_chain_A="AB", new_chain_B="CD")

In [ ]:
truncator.regroup_chains("testing_out//ZCON_37_3+1__numH4__from-22.25__to17.45.pdb", "testing_out/grouped",
                        new_chain_A="C", new_chain_B="ABD", save_segi="True")

In [ ]:
truncator.regroup_chains("testing_out//ZCON_37_3+1__numH4__from-22.25__to17.45.pdb", "testing_out/grouped",
                        new_chain_A="C", new_chain_B="ABD", save_segi=False,
                        out_name="testing_out/grouped/ZCON_37_3+1__numH4__from-22.25__to17.45__grC-ABD.nosegi.pdb")

In [ ]:
#cmd.load("testing_out//ZCON_37_3+1__numH4__from-22.25__to17.45.pdb")
#m = cmd.get_model("chain A")

In [ ]:
pdb_files = glob("testing_out/grouped/*.pdb")
pdb_files

In [ ]:
%%time
sf= []
for pdb in pdb_files:
    sf.append(truncator.score_interface(pdb,
                    script_name="truncator/xml/03_FD_FR_and_scoring_benchmark_v3_digs_updated_schema_BUNS3_IA_quick_no_holes.xml",
                    out_dir="testing_out/grouped/scoring", test_run=False))
sf

In [ ]:
import truncator
score_files = sorted(glob("testing_out/grouped/*_IA.sc"))
score_files

In [ ]:
scores = truncator.read_score_files(score_files, skiprows=False, load_seq=False).set_index('decoy')
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores.transpose())

In [ ]:
score

In [ ]:
files = sorted(glob("testing_out/grouped/*_IA*.sc"), reverse=True)
files

In [ ]:
scores_sasa = [col for col in scores.columns if "sasa" in col.lower()]
scores_sasa
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores[scores_sasa].transpose())

In [ ]:
scores_hbond = [col for col in scores.columns if "loop_sc" in col.lower()]
scores_hbond
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores[scores_hbond].transpose())

In [ ]:
truncator.score_interface("data/ZCON_37_N3_C1_2.pdb",
                script_name="truncator/xml/04_FD_FR_and_scoring_benchmark_v3_digs_updated_schema_BUNS3_IA_select.xml",
                out_dir="testing_out/grouped", tee=False, test_run=True)

In [ ]:

scores = truncator.read_score_files(['testing_out/grouped/ZCON_37_N3_C1_2_IA.sc'], skiprows=False, load_seq=False).set_index('description')
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores.transpose())

In [ ]:
interesting_cols = """
interface_averageDegree_chA
interface_averageDegree_chB
interface_buried_sasa
interface_contactcount
interface_fxn_hydrophobic
interface_hbonds
interface_holes
interface_hydrophobic_sasa
interface_polar_sasa
interface_salt_bridges
interface_sc
interface_sc_int_area
interface_unsat_hbond1
interface_unsat_hbond2
cross_interface_energy_IA
cross_interface_hbonds_IA
delta_unsat_hbonds_IA
hbond_energy_IA
interface_delta_sasa_IA
interface_hydrophobic_sasa_IA
interface_polar_sasa_IA
exposed_hydrophobics
exposed_polars
exposed_total
""".split()
#print("\n".join(list(df.columns)))
df[interesting_cols].transpose()

In [ ]:
import os
import subprocess, sys
os.system("echo TEST")
!ls -l

In [ ]:
pd.set_option('display.max_rows', None)
pd.options.display.max_columns=None
df.transpose()

In [ ]:
import dask
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(cores=1,
                     memory="2GB",
                     queue='interactive',)
cluster.scale(4)

from dask.distributed import Client
client = Client(cluster)